In [1]:
import os
import pdfplumber

data_folder = "E:\\Resume_Macher_Project\\data"
jd_pdf_path = os.path.join(data_folder, "jd.pdf")
with pdfplumber.open(jd_pdf_path) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text() + "\n"

resume_text = {}
for file in os.listdir(data_folder):
    if file.endswith(".pdf") and file != "jd.pdf":
        path = os.path.join(data_folder, file)
        resume_text[file] = "" 
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                resume_text[file] += page.extract_text() + "\n"
print("JD length:", len(text))
print("Total resumes extracted:", len(resume_text))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


JD length: 2395
Total resumes extracted: 7


//Data extracted from pdf

In [2]:
import os
ouput_folder = "E:\\Resume_Macher_Project\\output"
os.makedirs(ouput_folder, exist_ok=True)

with open(os.path.join(ouput_folder, "jd.txt"), "w", encoding="utf-8") as f:
    f.write(text.strip())
    
for filename, text in resume_text.items():
    with open(os.path.join(ouput_folder, filename.replace(".pdf", ".txt")), "w", encoding="utf-8") as f:
        f.write(text.strip())
print("JD and resumes have been extracted and saved to the output folder.")

JD and resumes have been extracted and saved to the output folder.


//Extracted text converted and saved as txt file

In [3]:
input_folder = "E:\\Resume_Macher_Project\\output"
output_folder = "E:\\Resume_Macher_Project\\output\\cleaned"
os.makedirs(output_folder, exist_ok=True)

In [4]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    doc = nlp(text)
    cleaned = " ".join([token.text for token in doc if not token.is_stop])
    return cleaned
    

In [5]:
for file in os.listdir(input_folder):
    if file.endswith(".txt"):
        with open(os.path.join(input_folder, file), "r", encoding="utf-8") as f:
            raw_text = f.read()
        cleaned = clean_text(raw_text)
        with open(os.path.join(output_folder, file), "w", encoding="utf-8") as f:
            f.write(cleaned)
print("Text cleaning completed. Cleaned files are saved in the cleaned folder.")

Text cleaning completed. Cleaned files are saved in the cleaned folder.


//txt file cleaned and saved

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-base-en')
print("Model loaded successfully.")

e:\Resume_Macher_Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully.


In [7]:
import os
input_folder = "E:\\Resume_Macher_Project\\output\\cleaned"
text = {}
for file in os.listdir(input_folder):
    if file.endswith(".txt"):
        with open(os.path.join(input_folder, file), "r", encoding="utf-8") as f:
            text[file] = f.read()
print("Loaded:", len(text),"cleaned text files")

Loaded: 8 cleaned text files


In [8]:
embeddings = {}
for filename, content in text.items():
    embeddings[filename] = model.encode(content, convert_to_tensor=True)
    print("embeddings created")

embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created


In [9]:
from sentence_transformers import util
jd_file = "jd.txt"
jd_embedding = embeddings[jd_file]
similarity_scores = {}
for file, emb in embeddings.items():
    if file != jd_file:
        score = util.pytorch_cos_sim(jd_embedding, emb).item()
        similarity_scores[file] = round(score * 100, 2)
sorted_scores = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
print("Top Resume Matches:")
for file, score in sorted_scores:
    print(f"{file}:{score}%match")

Top Resume Matches:
Resume1.txt:91.98%match
Resume2.txt:86.09%match
Resume7.txt:84.36%match
Resume4.txt:83.74%match
Resume6.txt:83.17%match
Resume3.txt:78.96%match
Resume5.txt:75.93%match


In [10]:
import spacy 
from collections import Counter
import re
nlp = spacy.load("en_core_web_sm")
def extract_keywords_from_jd(jd_text, top_k=10):
    doc = nlp(jd_text.lower())
    keywords = [token.text for token in doc
                if token.pos_ in ["NOUN", "PROPN"]
                and len(token.text) > 2
                and not token.is_stop
                and not token.is_punct]
    cleaned = [re.sub(r'[^a-zA-Z0-9\- ]', '', word) for word in keywords]
    freq = Counter(cleaned)
    return [word for word, count in freq.most_common(top_k)]

In [11]:
jd_text = text["jd.txt"] 
jd_keywords = extract_keywords_from_jd(jd_text, top_k=10)

print("Extracted Keywords:", jd_keywords)

Extracted Keywords: ['deployment', 'models', 'projects', 'data', 'model', 'experience', 'pipelines', 'job', 'engineer', 'hands']


In [12]:
from sentence_transformers import util
def match_resumes(jd_text, resume_texts, model, weight_semantics=0.4, weight_keyword=0.6):
    jd_keywords = extract_keywords_from_jd(jd_text, top_k=10)
    jd_embedding = model.encode(jd_text, convert_to_tensor=True)
    results = []
    for filename, text in resume_texts.items():
        resume_embedding = model.encode(text, convert_to_tensor=True)
        semantic_score = util.pytorch_cos_sim(jd_embedding, resume_embedding).item() * 100
        matched_keywords = sum([1 for kw in jd_keywords if kw.lower() in text.lower()])
        keyword_score = (matched_keywords / len(jd_keywords)) * 100

        final_score = weight_semantics * semantic_score + weight_keyword * keyword_score

        results.append({
            "filename": filename,
            "semantic_score": round(semantic_score, 2),
            "keyword_score": round(keyword_score, 2),
            "final_score": round(final_score, 2)
        })

    return sorted(results, key=lambda x: x["final_score"], reverse=True)

In [13]:
jd_text = text["jd.txt"]
resumes = {k: v for k, v in text.items() if k != "jd.txt"}

results = match_resumes(jd_text, resumes, model)
for r in results:
    print(r)



{'filename': 'Resume1.txt', 'semantic_score': 91.98, 'keyword_score': 80.0, 'final_score': 84.79}
{'filename': 'Resume2.txt', 'semantic_score': 86.09, 'keyword_score': 40.0, 'final_score': 58.43}
{'filename': 'Resume4.txt', 'semantic_score': 83.74, 'keyword_score': 40.0, 'final_score': 57.49}
{'filename': 'Resume7.txt', 'semantic_score': 84.36, 'keyword_score': 30.0, 'final_score': 51.75}
{'filename': 'Resume6.txt', 'semantic_score': 83.17, 'keyword_score': 20.0, 'final_score': 45.27}
{'filename': 'Resume3.txt', 'semantic_score': 78.96, 'keyword_score': 10.0, 'final_score': 37.58}
{'filename': 'Resume5.txt', 'semantic_score': 75.93, 'keyword_score': 0.0, 'final_score': 30.37}


In [14]:
import pandas as pd
df_results = pd.DataFrame(results)
df_results.to_csv("matching_results.csv", index=False)
print("Results saved to matching_results.csv")

Results saved to matching_results.csv
